<a href="https://colab.research.google.com/github/Ramubala/hindi-tokenizer/blob/main/Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle transformers[torch]
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 25.7 MB/s eta 0:00:00


Saving kaggle.json to kaggle.json


In [2]:
!kaggle datasets download -d vaibhavkumar11/hindi-english-parallel-corpus
!mkdir train
!unzip hindi-english-parallel-corpus.zip -d data

 98% 110M/112M [00:06<00:00, 29.3MB/s]
100% 112M/112M [00:06<00:00, 19.3MB/s]
Archive:  hindi-english-parallel-corpus.zip
  inflating: data/hindi_english_parallel.csv  


In [22]:
import pandas as pd
data = pd.read_csv('data/hindi_english_parallel.csv')
data = data[['hindi']]
data.drop_duplicates(inplace=True)

data.rename(columns={'hindi':'text'},inplace=True)
data['text'] = data['text'].astype(str).astype(pd.StringDtype()) 
data.head()

,text
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...


In [23]:
data.dtypes

text    string
dtype: object

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(data, test_size=0.20, random_state=42)

X_train.reset_index(inplace=True,drop=True)
X_test.reset_index(inplace=True,drop=True)

X_train.shape, X_test.shape

((787152, 1), (196788, 1))

In [25]:
X_train.text.dtype

string[python]

In [26]:
def get_training_corpus():
    for i in range(0, len(X_train), 1000):
        yield X_train[i : i + 1000]["text"]

In [27]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

# no normalizer is needed for hindi language


# pre tokenize using whitespaces
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# use the models to get subword tokenization
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens, min_frequency=5)

tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [29]:
tokenizer.save("tokenizer.json")

In [28]:
for i in range(10):
  encoding = tokenizer.encode(X_test.loc[i]['text'])
  print(X_test.loc[i]['text'])
  print(encoding.tokens)

पुनपुन नदी 
['पुन', '##पु', '##न', 'नदी']
पेंशन निधि ने किसी भी सहयोगी को पेंशन फंड के व्यापार से संबंधित किसी प्रकार की कोई अनावश्यक और अनुचित सुविधा तो नहीं दी है जो अभिदाताओं के हितों के लिए किसी भी प्रकार से हानिकारक हो। 
['पेंशन', 'निधि', 'ने', 'किसी', 'भी', 'सहयोगी', 'को', 'पेंशन', 'फंड', 'के', 'व्यापार', 'से', 'संबंधित', 'किसी', 'प्रकार', 'की', 'कोई', 'अनावश्यक', 'और', 'अनुचित', 'सुविधा', 'तो', 'नहीं', 'दी', 'है', 'जो', 'अभिदाता', '##ओं', 'के', 'हितों', 'के', 'लिए', 'किसी', 'भी', 'प्रकार', 'से', 'हानिकारक', 'हो', '।']
गुबरैला और चेफर भृंगों की अनेक जातियां भी यहां रहती हैं. 
['गु', '##बर', '##ैला', 'और', 'चे', '##फर', 'भृंग', '##ों', 'की', 'अनेक', 'जातियां', 'भी', 'यहां', 'रहती', 'हैं', '.']
अगर किसी पहचाने हुए यौन अपराधी के व्यवहार से आपका समुदाय चिंतित है, तो पुलिस शेद् औङ्ङेन्डेर् और्डेर् (सैक्स अऑफ़ैंडर अऑर्डर अर्थात यौन अपराधी आदेश) के लिए मेजिस्ट्रेट की अदालत को आवेदन दे सकती हे
['अगर', 'किसी', 'पहचाने', 'हुए', 'यौन', 'अपराधी', 'के', 'व्यवहार', 'से', 'आपका', 'समुदाय', 'चिं